In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0

# REQUIRED: Set to the model pathname
MODEL_PATH = "/root/model/"

# SageMaker endpoint requires a bundled model with a numerical version number as the directory name
MODEL_VERSION = "0001"

# Select which type of SageMaker EC2 instance to deploy the model on  
INSTANCE_TYPE = 'ml.c5.xlarge' 

# Specify the number of instances
INSTANCE_NUM = 1

# Specify the name for the SageMaker endpoint
ENDPOINT_NAME = "cell-segmentation-model"

In [ ]:
from sagemaker import get_execution_role
from sagemaker import Session
from sagemaker.tensorflow.serving import Model
from datetime import datetime
role = get_execution_role()
sess = Session()
bucket = sess.default_bucket()

import tensorflow as tf
import shutil
import os
import tarfile

print(tf.__version__)  # This notebook runs on TensorFlow 2.11.0 as tested for the workshop
tf_framework_version = tf.__version__

In [ ]:
# Display the default bucket associated to the SageMaker instance
print(bucket)

In [ ]:
model_bundle = MODEL_PATH + MODEL_VERSION

if os.path.exists(model_bundle) == False:
    shutil.copytree(MODEL_PATH + "unet-cell-segmentation-v2-512x512.model", model_bundle)

!saved_model_cli show --all --dir {model_bundle}

In [ ]:
model_archive = MODEL_PATH + 'cell-segmentation-model.tar.gz'

with tarfile.open(model_archive, mode='w:gz') as archive:
    archive.add(model_bundle, recursive=True) 

# upload model artifacts to S3
model_s3 = sess.upload_data(path=model_archive, key_prefix='data')

In [ ]:
print(model_s3)

In [ ]:
# Instantiate the SageMaker TensorFlow serving model  
sm_model = Model(model_data=model_s3, 
                 framework_version=tf_framework_version,
                 role=role)

# v2.0 and later
#Model("unet-cell-segmentation:latest", role=role, framework_version=tf_framework_version)
#, model_data="s3://sagemaker-ap-southeast-2-072766311530/model/model.tar.gz")

uncompiled_predictor = sm_model.deploy(endpoint_name=ENDPOINT_NAME, initial_instance_count=INSTANCE_NUM, instance_type=INSTANCE_TYPE)

In [ ]:
print("Successfully created SageMaker Endpoint => ", ENDPOINT_NAME)